In [1]:
!pip install xlrd


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '..'))


In [66]:
import json
import pandas as pd

In [10]:
DATA_PATH = "/mnt/c/Users/lschettini/OneDrive - Microsoft/Shared Documents - BabelBench for Healthcare/Demo data (internal only)/Chest CR/"
IMAGES_PATH = os.path.join(DATA_PATH, "matt10_chestcr_pa_lateral")
CSV_PATH = os.path.join(DATA_PATH, "laterals_with_priors_cohort_unique_studies.csv")

In [14]:
filenames = os.listdir(IMAGES_PATH)
filenames

['1.3.6.1.4.1.55648.003520501353637102596177915352393107475.1.1.dcm',
 '1.3.6.1.4.1.55648.003520501353637102596177915352393107475.2.1.dcm',
 '1.3.6.1.4.1.55648.048772281635966174841166266463101169808.1.1.dcm',
 '1.3.6.1.4.1.55648.048772281635966174841166266463101169808.1.2.dcm',
 '1.3.6.1.4.1.55648.065422838018990733185671226621328672106.1.1.dcm',
 '1.3.6.1.4.1.55648.065422838018990733185671226621328672106.2.1.dcm',
 '1.3.6.1.4.1.55648.072453241448818712689493486939468855621.1.1.dcm',
 '1.3.6.1.4.1.55648.072453241448818712689493486939468855621.2.1.dcm',
 '1.3.6.1.4.1.55648.108811176320215429875783676699432745124.1.1.dcm',
 '1.3.6.1.4.1.55648.108811176320215429875783676699432745124.2.1.dcm',
 '1.3.6.1.4.1.55648.151160455419332753189783378174892156836.1.1.dcm',
 '1.3.6.1.4.1.55648.151160455419332753189783378174892156836.1.7.dcm',
 '1.3.6.1.4.1.55648.151160455419332753189783378174892156836.1.8.dcm',
 '1.3.6.1.4.1.55648.221864164613020762982019272565694925423.1.1.dcm',
 '1.3.6.1.4.1.55648.

In [44]:
df = pd.read_csv(CSV_PATH)

df.shape, df.columns

((1138, 21),
 Index(['Report', 'Look at images?', 'ANATOMY', 'EXAMINATION', 'Modality',
        'StudyInstanceUID', 'SOPInstanceUID', 'ViewPosition', 'PatientID',
        'BodyPartExamined', 'Manufacturer', 'ManufacturerModelName',
        'StudyDate', 'AcquisitionTime', 'PatientSex', 'PatientAge',
        'BitsAllocated', 'PhotometricInterpretation', 'Rows', 'Columns',
        'FilePath'],
       dtype='object'))

In [45]:
df["Filename"] = df["FilePath"].str.split("/").str[-1]

In [46]:
df = df[df["Filename"].isin(filenames)].copy()

df.shape

(10, 22)

In [47]:
# do not truncate
with pd.option_context('display.max_colwidth', None):
    display(df["Report"])

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     CXR2V Procedure(s): XR chest 2V Chest two views Indication: Pleural effusion Reference: segmed_MM/DD/YYYY Findings: Moderate to large right pleural effusion has not changed significantly in the interval. Mildly tortuous aorta. Right heart border is obscured. No emphysema. Mild scarring or subsegmental atelectasis the left lung base. Pas

In [48]:
key_labels = {
    "indication": [
        "Indication:",
        "CHEST 2 VIEWS Indication :",
        "Clinical information:",
        "CHEST 2 VIEWS History:",
        "History:",
    ],
    "findings": [
        "Findings:",
    ],
    "technique": [
        "Technique:",
    ],
    "comparison": [
        "Comparison:",
    ],
    "procedure": [
        "CXR2V Procedure(s):",
    ],
    "examination": [
        "Examination:",
    ],
    "impression": [
        "Impression:",
    ],
}


def get_label_contents(report, key_labels):
    report = report.replace("\n", " ")
    for key, labels in key_labels.items():
        for label in labels:
            report = report.replace(label, f"\n{key}: ")

    label_content = {}
    for line in report.split("\n"):
        for key in key_labels.keys():
            if line.startswith(f"{key}:"):
                if key not in label_content:
                    label_content[key] = []
                label_content[key].append(line.replace(f"{key}:", "").strip())

    return label_content


df["processed_report"] = df["Report"].apply(
    get_label_contents, key_labels=key_labels
)

df[["processed_report", "Report"]]

,processed_report,Report
0,"{'procedure': ['XR chest 2V Chest two views'],...",CXR2V Procedure(s): XR chest 2V Chest two view...
6,{'indication': ['Lung cancer. Compared to segm...,CHEST 2 VIEWS Two view chest History: Lung can...
119,"{'procedure': ['XR chest 2V'], 'examination': ...",CXR2V Procedure(s): XR chest 2V Examination: C...
134,{'indication': ['Six weeks post pneumonia. Sho...,CHEST 2 VIEWS Indication : Six weeks post pneu...
140,"{'indication': ['Status post thoracentesis.'],...",CHEST 2 VIEWS Indication : Status post thorace...
168,"{'procedure': ['XR chest 2V'], 'indication': [...",CXR2V Procedure(s): XR chest 2V Indication: Lu...
706,"{'indication': ['PERSISTENT COUGH.'], 'compari...",CHEST 2 VIEWS 2 View Chest History:PERSISTENT ...
723,"{'indication': ['Followup pneumonia.'], 'techn...",CHEST 2 VIEWS Indication : Followup pneumonia....
825,"{'indication': ['follow-up for pneumonia'], 'c...",CHEST 2 VIEWS History: follow-up for pneumonia...
861,{'indication': ['Hemoptysis Comparison segmed_...,CHEST 2 VIEWS Two-view chest segmed_MM/DD/YYYY...


In [51]:
for report in df["processed_report"]:
    for key, value in report.items():
        print(key, value)
        if len(value) > 1:
            print("WARNING: multiple values found")
    print()

procedure ['XR chest 2V Chest two views']
indication ['Pleural effusion Reference: segmed_MM/DD/YYYY']
findings ['Moderate to large right pleural effusion has not changed significantly in the interval. Mildly tortuous aorta. Right heart border is obscured. No emphysema. Mild scarring or subsegmental atelectasis the left lung base. Passive atelectasis in the right lung base. Degenerative changes in the spine.']

indication ['Lung cancer. Compared to segmed_MM/DD/YYYY, coarse linear and reticular opacities within the right base have worsened. Overall size of the abnormality is about the same but increased density is evident. Right upper lung is clear. Minor opacity in the left base appears similar. Lungs are otherwise unchanged. There is a mild kyphosis. Cannot exclude small amounts of pleural fluid.']
impression ['Worsening infiltrate in the right base. No significant change otherwise.']

procedure ['XR chest 2V']
examination ['Chest PA and lateral']
indication ['Followup pleural effusi

In [71]:
batch_review_json = {}
image_sample_json = []
for row in df.itertuples():
    indication = row.processed_report.get("indication", [""])[0]
    findings = row.processed_report.get("findings", [""])[0]

    if not indication:
        continue

    image_sample_json.append({
        "url": f"/dicom_images/demo/{row.Filename}",
        "filename": row.Filename,
        "indication": indication,
    })

    if not findings:
        continue

    xray_index = 0
    patient_xray_id = f"{row.PatientID}_{xray_index}"
    while patient_xray_id in batch_review_json:
        xray_index += 1
        patient_xray_id = f"{row.PatientID}_{xray_index}"
    

    batch_review_json[patient_xray_id] = {
        "indication": indication,
        "images": {
            "primary": f"https://imageviewersa.blob.core.windows.net/inference/demo_samples/{row.Filename}",
        },
        "groundTruth": [f"{f.strip()}." for f in findings.split(".") if f],
        "prediction": {
            "modelInference": [],
            "additionalInfo": "CXRReportGen",
        },
        "patientInfo": f"Patient is a {'female' if row.PatientSex == 'F' else 'male'} of {row.PatientAge[1:-1]} years old.",
    }


In [72]:
print(json.dumps(batch_review_json, indent=2))

{
  "Segmed_Patient_0765256604260653116_0": {
    "indication": "Pleural effusion Reference: segmed_MM/DD/YYYY",
    "images": {
      "primary": "https://imageviewersa.blob.core.windows.net/inference/demo_samples/1.3.6.1.4.1.55648.003520501353637102596177915352393107475.1.1.dcm"
    },
    "groundTruth": [
      "Moderate to large right pleural effusion has not changed significantly in the interval.",
      "Mildly tortuous aorta.",
      "Right heart border is obscured.",
      "No emphysema.",
      "Mild scarring or subsegmental atelectasis the left lung base.",
      "Passive atelectasis in the right lung base.",
      "Degenerative changes in the spine."
    ],
    "prediction": {
      "modelInference": [],
      "additionalInfo": "CXRReportGen"
    },
    "patientInfo": "Patient is a female of 85 years old."
  },
  "Segmed_Patient_3466228512147573455_0": {
    "indication": "Followup pleural effusion post thoracentesis",
    "images": {
      "primary": "https://imageviewersa.b

In [68]:
print(json.dumps(image_sample_json, indent=2))

[
  {
    "url": "/dicom_images/demo/1.3.6.1.4.1.55648.003520501353637102596177915352393107475.1.1.dcm",
    "filename": "1.3.6.1.4.1.55648.003520501353637102596177915352393107475.1.1.dcm",
    "indication": "Pleural effusion Reference: segmed_MM/DD/YYYY"
  },
  {
    "url": "/dicom_images/demo/1.3.6.1.4.1.55648.287343534839562161459062219490368751449.1.1.dcm",
    "filename": "1.3.6.1.4.1.55648.287343534839562161459062219490368751449.1.1.dcm",
    "indication": "Lung cancer. Compared to segmed_MM/DD/YYYY, coarse linear and reticular opacities within the right base have worsened. Overall size of the abnormality is about the same but increased density is evident. Right upper lung is clear. Minor opacity in the left base appears similar. Lungs are otherwise unchanged. There is a mild kyphosis. Cannot exclude small amounts of pleural fluid."
  },
  {
    "url": "/dicom_images/demo/1.3.6.1.4.1.55648.337129350405985376957945329162257509197.1.1.dcm",
    "filename": "1.3.6.1.4.1.55648.337129